Note:
1. "AAAA"-dict, "AaAa"-array, "Aaaa"-df, "aa_aa"-list, "aaaa"-single value.  
2. Stage - ID: Rest1 - 1, Cool Test - 2, Recover - 3, Rest2 - 4.
3. SIG array shape: ECG, CBP, PPG, IPG, Temp, Stage, ECGf, 

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import scipy.io
import os

### Read the signal data

In [2]:
data_root = 'C:\COCHE_Project\MMBP\data\MMData'
## The list of subject number.
sub_id = list(os.walk(data_root))[0][1]

In [3]:
def combine_row_signals(subid): ## Input "ID", Return EntireStage.
    ## Find signal segments in each stage.
    rest1_s = []
    CT_s = []
    recover_s = []
    rest2_s = []
    seg_signal_name = [n.split('.')[0] for n in list(os.walk(f"{data_root}\{subid}"))[0][2]]
    for segsig in seg_signal_name:
        SegSignal = scipy.io.loadmat(f"{data_root}\{subid}\{segsig}.mat")[segsig]
        for stage in segsig.split('_'):
            if stage in ['recover', 're', 'Recover']: #this seg is one of Recover
                recover_s.append(SegSignal)
            elif stage in ['rest1']:
                rest1_s.append(SegSignal)
            elif stage in ['CT', 'ct', 'ct1', 'ct2']: ##
                CT_s.append(SegSignal)
            elif stage in ['rest2']:
                rest2_s.append(SegSignal)
            elif stage in ['rest']:
                rest_index = segsig.split('_').index('rest')
                if segsig.split('_')[rest_index + 1] == '1':
                    rest1_s.append(SegSignal)
                else:
                    rest2_s.append(SegSignal)

    ## Concat each stage to one signal, and add Stage Tag under each signal array.  
    stageid = 0
    entire_stage = []
    for stage_list in [rest1_s, CT_s, recover_s, rest2_s]:       
        stageid += 1
        StageSig = np.concatenate(stage_list, axis=1)
        StageCom = np.concatenate((StageSig, np.full((1, StageSig.shape[1]), stageid)), axis=0)      
        entire_stage.append(StageCom)
    EntireStage = np.concatenate(entire_stage, axis=1)

    return EntireStage

In [4]:
SIG = {subid:combine_row_signals(subid) for subid in sub_id}

Show signals and save

### Process signals

In [78]:
fs = 2000
result_root = 'C:\COCHE_Project\MMBP\Result\Result_12Dec23'
subid = '1'

Filter signals

In [7]:
def filter_ecg(Sig, lowcut=10.0, highcut=50.0, order=4, figshow=0):
    b,a = signal.butter(order, [lowcut, highcut], btype='band', fs=fs)
    ecg_filtered = signal.filtfilt(b, a, Sig)

    if figshow == 1 :
        IntervalIndex = np.append(np.arange(0, len(Sig), 50000), len(Sig))
        for interind in range(len(IntervalIndex) - 1):
            begin_index = IntervalIndex[interind]
            end_index = IntervalIndex[interind + 1]
            fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(18,4),dpi=96)
            axs[0].plot(Sig[begin_index:end_index])
            axs[0].set_title('Original ECG')
            axs[1].plot(ecg_filtered[begin_index:end_index])
            axs[1].set_title('Filtered ECG')
            fig.subplots_adjust(hspace=0.55)
            fig.suptitle(f"S{subid} {int(begin_index/fs)} to {int(end_index/fs)} sec", fontsize=16)
            result_path = result_root + f"\S{subid}\ECG"
            if not os.path.exists(result_path):
                os.makedirs(result_path)
            plt.savefig(f"{result_path}\S{subid}_Ori&Fil_ECG_{int(begin_index/fs)}to{int(end_index/fs)}sec.png")
            plt.show()

    return ecg_filtered


In [12]:
## Add ECGf to the end of SIG array
SIG[subid] = combine_row_signals(subid)
print(SIG[subid].shape)
SIG[subid] = np.concatenate((SIG[subid], filter_ecg(SIG[subid][0], figshow=0).reshape(1, -1)), axis=0)
SIG[subid].shape

(6, 888006)


(7, 888006)

Find keypoints

In [83]:
def find_Rpeak(Sig, r_height=0.5, beat=100, figshow=0):
    ## Find peaks once in a window length, one by one.
    rpeak_index = []
    rpeak_value = []
    IntervalIndex = np.append(np.arange(0, len(Sig), 100000), len(Sig))
    for interind in range(len(IntervalIndex) - 1):
        begin_index = IntervalIndex[interind]
        end_index = IntervalIndex[interind + 1]
        SigSeg = Sig[begin_index: end_index]
        height = r_height * np.max(SigSeg)
        distance = 60 / beat * fs
        rpeak_index_seg, rpeak_value_fil = signal.find_peaks(SigSeg, height=height, distance=distance)
        rpeak_index.append(rpeak_index_seg)
        rpeak_value.append(rpeak_value_fil['peak_heights'])

        if figshow == 1:
            plt.figure(figsize=(18,3),dpi=96)
            plt.plot(SigSeg)
            plt.scatter(rpeak_index_seg, rpeak_value_fil['peak_heights'])
            plt.title(f"S{subid} R-peak from {int(begin_index/fs)} to {int(end_index/fs)} sec")
            result_path = result_root + f"\S{subid}\Rpeak"
            if not os.path.exists(result_path):
                os.makedirs(result_path)
            plt.savefig(f"{result_path}\S{subid}_Rpeak_{int(begin_index/fs)}to{int(end_index/fs)}sec.png")
            plt.show()

    #print(rpeak_value, rpeak_index)
    RpeakIndex = np.concatenate(rpeak_index).reshape(1, -1)
    RpeakValue = np.concatenate(rpeak_value).reshape(1, -1)
    return np.concatenate((RpeakIndex, RpeakValue), axis=0)

In [115]:
def find_keypoints(subid, r_height=0.5, beat=100, figshow=0):
    print(f"Subject {subid}")
    ECGSig = SIG[subid][6]
    CBPSig = SIG[subid][1]
    PPGSig = SIG[subid][2]
    IPGSig = SIG[subid][3]

    ## Find R peaks
    rpeak_index = []
    rpeak_value = []
    IntervalIndex = np.append(np.arange(0, len(ECGSig), 60*fs), len(ECGSig))
    for interind in range(len(IntervalIndex) - 1):
        begin_index = IntervalIndex[interind]
        end_index = IntervalIndex[interind + 1]
        ECGSeg = ECGSig[begin_index: end_index]
        height = r_height * np.max(ECGSeg)
        distance = 60 / beat * fs
        RpeakIndexSeg, rpeak_value_fil = signal.find_peaks(ECGSeg, height=height, distance=distance)
        rpeak_index.append(RpeakIndexSeg + begin_index)
        rpeak_value.append(rpeak_value_fil['peak_heights'])
    RpeakIndex = np.concatenate(rpeak_index)
    RpeakValue = np.concatenate(rpeak_value)
    #print(RpeakIndex)

    ## Find keypoints in other signals
    sbp_index = []
    dbp_index = []
    maxppg_index = []
    minppg_index = []
    highipg_index = []
    lowipg_index = []
    ipg_area = []
    SigSegIndex = np.append(RpeakIndex, len(ECGSig))
    #print(SigSegIndex.shape)
    for segint in range(len(SigSegIndex) - 1):
        begin_index = SigSegIndex[segint]
        end_index = SigSegIndex[segint + 1]
        
        ## Find SBP and DBP
        CBPSeg = CBPSig[begin_index: end_index]
        #print(CBPSeg)
        sbp_i, _ = signal.find_peaks(CBPSeg)
        if len(sbp_i) > 0:
            SbpV = CBPSeg[sbp_i]
            sbp_index.append(sbp_i[np.argmax(SbpV)] + begin_index)
            ## DBP between R peak and SBP
            HCBPSeg = CBPSig[begin_index: sbp_index[-1]]
            dbp_i, _ = signal.find_peaks(-HCBPSeg)
            if len(dbp_i) > 0:
                DbpV = HCBPSeg[dbp_i]
                dbp_index.append(dbp_i[np.argmin(DbpV)] + begin_index)
            else:
                dbp_index.append(sbp_index[-1])
        else:
            sbp_index.append(begin_index)
            dbp_index.append(begin_index)

        ## Find MaxPPG and MinPPG
        PPGSeg = PPGSig[begin_index: end_index]
        maxppg_i, _ = signal.find_peaks(PPGSeg)
        if len(maxppg_i) > 0:
            MaxppgV = PPGSeg[maxppg_i]
            maxppg_index.append(maxppg_i[np.argmax(MaxppgV)] + begin_index)
            ## MinPPG between R peak and MaxPPG
            HPPGSeg = PPGSig[begin_index: maxppg_index[-1]]
            minppg_i, _ = signal.find_peaks(-HPPGSeg)
            if len(minppg_i) > 0:
                MinppgV = HPPGSeg[minppg_i]
                minppg_index.append(minppg_i[np.argmin(MinppgV)] + begin_index)
            else:
                minppg_index.append(maxppg_index[-1])
        else:
            maxppg_index.append(begin_index)
            minppg_index.append(begin_index)

        ## Find LowIPG, HighIPG, and IPG area.
        IPGSeg = IPGSig[begin_index: end_index]
        if np.argmin(IPGSeg) > 0:
            lowipg_index.append(np.argmin(IPGSeg) + begin_index)
            HIPGSeg = IPGSig[begin_index: lowipg_index[-1]]
            highipg_i, _ = signal.find_peaks(HIPGSeg)
            if len(highipg_i) > 0:
                HighipgV = HIPGSeg[highipg_i]
                highipg_index.append(highipg_i[np.argmax(HighipgV)] + begin_index)
            else:
                highipg_index.append(np.argmax(HIPGSeg) + begin_index)
        else:
            lowipg_index.append(begin_index)
            highipg_index.append(begin_index)
        XV = np.arange(len(IPGSeg))
        area = np.trapz(IPGSeg, XV)
        ipg_area.append(area / fs)

    print(f"SBP index: {len(sbp_index)}, {sbp_index}\nDBP index: {len(dbp_index)}, {dbp_index}")
    print(f"Max PPG index: {len(maxppg_index)}, {maxppg_index}\nMin PPG index: {len(minppg_index)}, {minppg_index}")
    print(f"High IPG index: {len(highipg_index)}, {highipg_index}\nLow IPG index: {len(lowipg_index)}, {lowipg_index}")
    print(f"IPG_area: {len(ipg_area)}, {ipg_area}")

    SBPIndex = np.array(sbp_index)
    SBPValue = CBPSig[SBPIndex]
    DBPIndex = np.array(dbp_index)
    DBPValue = CBPSig[DBPIndex]
    MaxPPGIndex = np.array(maxppg_index)
    MaxPPGValue = PPGSig[MaxPPGIndex]
    MinPPGIndex = np.array(minppg_index)
    MinPPGValue = PPGSig[MinPPGIndex]
    HighIPGIndex = np.array(highipg_index)
    HighIPGValue = IPGSig[HighIPGIndex]
    LowIPGIndex = np.array(lowipg_index)
    LowIPGValue = IPGSig[LowIPGIndex]
    IPGArea = np.array(ipg_area)

    Keypoints = pd.DataFrame({
        'RpeakI': RpeakIndex, 'RpeakV': RpeakValue, 
        'SBPI': SBPIndex, 'SBPV': SBPValue, 'DBPI': DBPIndex, 'DBPV': DBPValue,
        'MaxPPGI': MaxPPGIndex, 'MaxPPGV': MaxPPGValue, 'MinPPGI': MinPPGIndex, 'MinPPGV': MinPPGValue, 
        'HighIPGI': HighIPGIndex, 'HighIPGV': HighIPGValue, 'LowIPGI': LowIPGIndex, 'LowIPGV': LowIPGValue, 'IPGArea': IPGArea
        })

    display(Keypoints)

    if figshow == 1:
        for interind in range(len(IntervalIndex) - 1):
            begin_index = IntervalIndex[interind]
            end_index = IntervalIndex[interind + 1]
            fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(18,8),dpi=96)
            axs[0].plot(ECGSig[begin_index:end_index])
            RpeakISeg = RpeakIndex[(RpeakIndex >= begin_index) & (RpeakIndex <= end_index)]
            RpeakVSeg = ECGSig[RpeakISeg]
            axs[0].scatter((RpeakISeg - begin_index), RpeakVSeg)
            axs[0].set_title('R peak in ECG')

            axs[1].plot(CBPSig[begin_index:end_index])
            SBPISeg = SBPIndex[(SBPIndex >= begin_index) & (SBPIndex <= end_index)]
            SBPVSeg = CBPSig[SBPISeg]
            axs[1].scatter((SBPISeg - begin_index), SBPVSeg)
            DBPISeg = DBPIndex[(DBPIndex >= begin_index) & (DBPIndex <= end_index)]
            DBPVSeg = CBPSig[DBPISeg]
            axs[1].scatter((DBPISeg - begin_index), DBPVSeg)
            axs[1].set_title('SBP & DBP in CBP')

            axs[2].plot(PPGSig[begin_index:end_index])
            MaxPPGISeg = MaxPPGIndex[(MaxPPGIndex >= begin_index) & (MaxPPGIndex <= end_index)]
            MaxPPGVSeg = PPGSig[MaxPPGISeg]
            axs[2].scatter((MaxPPGISeg - begin_index), MaxPPGVSeg)
            MinPPGISeg = MinPPGIndex[(MinPPGIndex >= begin_index) & (MinPPGIndex <= end_index)]
            MinPPGVSeg = PPGSig[MinPPGISeg]
            axs[2].scatter((MinPPGISeg - begin_index), MinPPGVSeg)
            axs[2].set_title('Max & Min in PPG')

            axs[3].plot(IPGSig[begin_index:end_index])
            HighIPGISeg = HighIPGIndex[(HighIPGIndex >= begin_index) & (HighIPGIndex <= end_index)]
            HighIPGVSeg = IPGSig[HighIPGISeg]
            axs[3].scatter((HighIPGISeg - begin_index), HighIPGVSeg)
            LowIPGISeg = LowIPGIndex[(LowIPGIndex >= begin_index) & (LowIPGIndex <= end_index)]
            LowIPGVSeg = IPGSig[LowIPGISeg]
            axs[3].scatter((LowIPGISeg - begin_index), LowIPGVSeg)
            axs[3].set_title('High & Low in IPG')

            fig.subplots_adjust(hspace=0.55)
            fig.suptitle(f"S{subid} {int(begin_index/fs)} to {int(end_index/fs)} sec", fontsize=16)
            result_path = result_root + f"\S{subid}\Keypoint\OriKey"
            if not os.path.exists(result_path):
                os.makedirs(result_path)
            plt.savefig(f"{result_path}\S{subid}_OriKey_{int(begin_index/fs)}to{int(end_index/fs)}sec.png")
            plt.show()
            
    result_path = result_root + f"\S{subid}\Keypoint\OriKey"
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    Keypoints.to_csv(f"{result_path}\S{subid}_Keypoints.csv", index=False)

    return Keypoints


In [116]:
Keypoints = find_keypoints(subid, figshow=0)

Subject 1
SBP index: 557, [3405, 4984, 6584, 8084, 9572, 11148, 12764, 14436, 16004, 17472, 19030, 20638, 22250, 23848, 25292, 26823, 28402, 30032, 31567, 33087, 34688, 36314, 37902, 39366, 40868, 42417, 43995, 45452, 46905, 48412, 49972, 51531, 52996, 54520, 56072, 57627, 59244, 60689, 62176, 63718, 65284, 66836, 68257, 69722, 71240, 72770, 74239, 75659, 77145, 78697, 80263, 81759, 83166, 84660, 86155, 87695, 89112, 90503, 92000, 93586, 95221, 96889, 98314, 99834, 101430, 103052, 104607, 106067, 107593, 109208, 110758, 112250, 113779, 115353, 116954, 118465, 119969, 120906, 122533, 123993, 125458, 127065, 128629, 130190, 131748, 133432, 135160, 136899, 138497, 140207, 141979, 143800, 145479, 147126, 148854, 150618, 152283, 153823, 155502, 157181, 158899, 160556, 162126, 163796, 165457, 167105, 168639, 170094, 171604, 173167, 174780, 176356, 177860, 179481, 181129, 182827, 184452, 186012, 187691, 189352, 190979, 192472, 194098, 195675, 197358, 198882, 200416, 202018, 203721, 205433, 20

,RpeakI,RpeakV,SBPI,SBPV,DBPI,DBPV,MaxPPGI,MaxPPGV,MinPPGI,MinPPGV,HighIPGI,HighIPGV,LowIPGI,LowIPGV,IPGArea
0,2776,0.097175,3405,125.640740,3251,73.028484,3605,2.631662,3157,-0.526629,3064,44.366942,3289,44.341604,34.997360
1,4355,0.098778,4984,127.715934,4804,75.469888,5181,2.246744,4744,-0.629308,4628,44.376969,4863,44.353832,34.871698
2,5928,0.098724,6584,126.556267,6424,78.918371,6744,2.012179,6343,-0.295080,6235,44.383322,6449,44.355027,33.189263
3,7425,0.085564,8084,124.084345,7910,82.366854,8215,1.887730,7719,0.131710,7734,44.382474,7935,44.358280,33.415740
4,8932,0.077317,9572,125.396600,9412,79.620275,9726,2.060073,9311,0.163667,9215,44.391909,9431,44.358748,35.394011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,881511,0.089969,882186,114.440799,881995,71.319501,882376,2.769006,881777,-0.616022,881822,45.253131,882151,45.230310,35.173506
553,883067,0.093092,883742,111.297492,883563,73.944010,883935,2.423905,883386,-0.464384,883382,45.251622,883634,45.231234,33.251892
554,884538,0.094304,885207,117.736695,885024,73.150554,885399,2.546626,884951,-0.292961,884858,45.252304,885065,45.230076,35.831907
555,886123,0.075057,886763,123.779170,886572,71.350019,886993,2.917243,886394,-0.449380,886431,45.262264,886650,45.233080,36.875673


Get features

In [ ]:
def get_features(subid, Keypoints):
    




























    features = {}
    features['Stage'] = Sub[f"{Sub['sub_id'][i]}"]['Stage'][r_peak_index[:-1]]
    features['R_peak_index'] = r_peak_index[:-1]
    print(f"len R_peak_index:{len(features['R_peak_index'])}")
    features['heart_rate'] = [(r_peak_index[rpi+1] - r_peak_index[rpi])/fs for rpi in range(len(r_peak_index)-1)]
    print(f"max HR:{max(features['heart_rate'])}, min HR:{min(features['heart_rate'])}, len HR:{len(features['heart_rate'])}")
    features['R_peak'] = Sub[f"{Sub['sub_id'][i]}"]['ECGf'][r_peak_index[:-1]]
    print(f"max R_peak:{max(features['R_peak'])}, min R_peak:{min(features['R_peak'])}, len R_peak:{len(features['R_peak'])}")
    features['PTT'] = [(max_ppg_index[mpi] - r_peak_index[mpi]) / fs for mpi in range(len(max_ppg_index))]
    print(f"max PTT:{max(features['PTT'])}, min PTT:{min(features['PTT'])}, len PTT:{len(features['PTT'])}")
    features['Max_PPG'] = Sub[f"{Sub['sub_id'][i]}"]['PPG'][max_ppg_index]
    print(f"max Max_PPG:{max(features['Max_PPG'])}, min Max_PPG:{min(features['Max_PPG'])}, len Max_PPG:{len(features['Max_PPG'])}")
    features['Min_PPG'] = Sub[f"{Sub['sub_id'][i]}"]['PPG'][min_ppg_index]
    print(f"max Min_PPG:{max(features['Min_PPG'])}, min Min_PPG:{min(features['Min_PPG'])}, len Min_PPG:{len(features['Min_PPG'])}")
    features['dPPG'] = features['Max_PPG'] - features['Min_PPG']
    print(f"max dPPG:{max(features['dPPG'])}, min dPPG:{min(features['dPPG'])}, len dPPG:{len(features['dPPG'])}")
    features['PPG_width'] = [(min_ppg_index[mpi+1] - min_ppg_index[mpi])/fs for mpi in range(len(min_ppg_index) - 1)]
    features['PPG_width'].append((r_peak_index[-1] - min_ppg_index[-1])/fs)
    print(f"max PPG_width:{max(features['PPG_width'])}, min PPG_width:{min(features['PPG_width'])}, len PPG_width:{len(features['PPG_width'])}")
    features['High_IPG'] = Sub[f"{Sub['sub_id'][i]}"]['IPG'][high_ipg_index]
    print(f"max High_IPG:{max(features['High_IPG'])}, min High_IPG:{min(features['High_IPG'])}, len High_IPG:{len(features['High_IPG'])}")
    features['Low_IPG'] = Sub[f"{Sub['sub_id'][i]}"]['IPG'][low_ipg_index]
    print(f"max Low_IPG:{max(features['Low_IPG'])}, min Low_IPG:{min(features['Low_IPG'])}, len Low_IPG:{len(features['Low_IPG'])}")
    features['dIPG'] = features['High_IPG'] - features['Low_IPG']
    print(f"max dIPG:{max(features['dIPG'])}, min dIPG:{min(features['dIPG'])}, len dIPG:{len(features['dIPG'])}")
    '''features['IPG_small_area'] = []
    features['IPG_large_area'] = []
    max_ipg = max(Sub[f"{Sub['sub_id'][i]}"]['IPG'])
    for rpi in range(len(r_peak_index) - 1):
        ipg_seg = Sub[f"{Sub['sub_id'][i]}"]['IPG'][r_peak_index[rpi]:r_peak_index[rpi+1]]
        features['IPG_small_area'].append(sum([(max(ipg_seg) - ipg_seg[d]) for d in range(len(ipg_seg))]) / fs)
        features['IPG_large_area'].append(sum([(max_ipg - ipg_seg[d]) for d in range(len(ipg_seg))]) / fs)
    print(f"max IPG_small_area:{max(features['IPG_small_area'])}, min IPG_small_area:{min(features['IPG_small_area'])}, len IPG_small_area:{len(features['IPG_small_area'])}")   
    print(f"max IPG_large_area:{max(features['IPG_large_area'])}, min IPG_large_area:{min(features['IPG_large_area'])}, len IPG_large_area:{len(features['IPG_large_area'])}")'''
    features['IPG_area'] = ipg_area
    print(f"max ipg_area:{max(features['IPG_area'])}, min ipg_area:{min(features['IPG_area'])}, len ipg_area:{len(features['IPG_area'])}")
    features['ave_Temp'] = [np.mean(Sub[f"{Sub['sub_id'][i]}"]['Temp'][r_peak_index[rpi]: r_peak_index[rpi+1]]) for rpi in range(len(r_peak_index) - 1)]
    print(f"max ave_Temp:{max(features['ave_Temp'])}, min ave_Temp:{min(features['ave_Temp'])}, len ave_Temp:{len(features['ave_Temp'])}")
    features['HR_PPG'] = [(max_ppg_index[mpi+1] - max_ppg_index[mpi])/fs for mpi in range(len(max_ppg_index)-1)]
    features['HR_PPG'].append(features['HR_PPG'][-1])
    print(f"max HR_PPG:{max(features['HR_PPG'])}, min HR_PPG:{min(features['HR_PPG'])}, len HR_PPG:{len(features['HR_PPG'])}")
    features['HR_IPG_high'] = [(high_ipg_index[hii+1] - high_ipg_index[hii])/fs for hii in range(len(high_ipg_index)-1)]
    features['HR_IPG_high'].append(features['HR_IPG_high'][-1])
    print(f"max HR_IPG_high:{max(features['HR_IPG_high'])}, min HR_IPG_high:{min(features['HR_IPG_high'])}, len HR_IPG_high:{len(features['HR_IPG_high'])}")
    features['HR_IPG_low'] = [(low_ipg_index[lii+1] - low_ipg_index[lii])/fs for lii in range(len(low_ipg_index)-1)]
    features['HR_IPG_low'].append(features['HR_IPG_low'][-1])
    print(f"max HR_IPG_low:{max(features['HR_IPG_low'])}, min HR_IPG_low:{min(features['HR_IPG_low'])}, len HR_IPG_low:{len(features['HR_IPG_low'])}")
    
    features['SBP'] = Sub[f"{Sub['sub_id'][i]}"]['CBP'][sbp_index]
    print(f"max SBP:{max(features['SBP'])}, min SBP:{min(features['SBP'])}, len SBP:{len(features['SBP'])}")
    features['DBP'] = Sub[f"{Sub['sub_id'][i]}"]['CBP'][dbp_index]
    print(f"max DBP:{max(features['DBP'])}, min DBP:{min(features['DBP'])}, len DBP:{len(features['DBP'])}")
    features['PBP'] = features['SBP'] - features['DBP']
    print(f"max PBP:{max(features['PBP'])}, min PBP:{min(features['PBP'])}, len PBP:{len(features['PBP'])}")
    
    features_df = pd.DataFrame(features)
    result_path = 'C:\COCHE_Project\MMBP\Result\Features\Original_Features'
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    features_df.to_csv(f"{result_path}\S{Sub['sub_id'][i]}_OriginalFeatures.csv", index=False)
    #display(features_df)


### Build models

LSTM model

RF model

### Validate models

Reshape input data

Test model

Show the results